In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, LSTM
import core.preprocessing as pp

LOGDIR = "/tmp/forecasting/"

present_window = 5*12
future_window = 3*12

# Load data
df = pd.read_pickle('data/detrended_master_dataset.pkl').fillna(0)

# Drop columns originally filled with NaNs
for item in pp.DROP_LIST:
    key = 'S{:02}'.format(item)
    del df[key]

nstamps = df.shape[0]
nseries = df.shape[1] - 1

# Split into training and dev
dftrain = pp.make_small_train(df, 'combined')
dfdev = pp.make_small_dev(df)

# Define how to produce batches
def make_batch(data, size):
    features = data.iloc[:,0:-1] # assume inverters are in columns 2, 3, ..., n-1
    response = data.iloc[:,-1] # assume aggregate power is in column n
    
    n = data.shape[0]
    
    X = []; Y = []
    for i in range(size):
        t = np.random.randint(n - present_window - future_window)
        x = features[t:t+present_window].values.T.flatten().tolist()
        y = response[t+present_window:t+present_window+future_window].values.tolist()
        X.append(x)
        Y.append(y)
    return np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)

def center_batch(data):
    X = data[0]
    Y = data[1]
    m = np.mean(X, 0)
    s = np.std(X, 0)
    Xcent = (X - m) / s
    m = np.mean(Y, 0)
    s = np.std(Y, 0)
    Ycent = (Y - m) / s
    return Xcent, Ycent

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
class FC(object):
    def __init__(self, layer_units):
        nlayers = len(layer_units)
        self.layers = []
        for i in range(nlayers):
            self.layers.append(Dense(layer_units[i], activation=tf.nn.relu))
        self.layers.append(Dense(future_window))
        
    def __call__(self, x):
        a = x
        for l in self.layers:
            a = l(a)
        return a

class CNN(object):
    def __init__(self, layer_units):
        nlayers = len(layer_units)
        self.layers = []
        for i in range(nlayers):
            self.layers.append(Conv2D(layer_units[i], kernel_size=3, activation=tf.nn.relu, padding='same'))
            self.layers.append(MaxPool2D(padding='same'))
        self.layers.append(Flatten())
        self.layers.append(Dense(future_window))
        
    def __call__(self, x):
        a = tf.reshape(x, [-1, nseries, present_window, 1])
        for l in self.layers:
            a = l(a)
        return a
    
class RNN(object):
    def __init__(self):
#         self.streams = []
        self.layer = LSTM(future_window, input_shape=(future_window, nseries))
#         for i in range(nseries):
#             stream = []
#             stream.append(LSTM(future_window))
#             self.streams.append(stream)
    
    def __call__(self, x):
        x_img = tf.reshape(x, [-1, nseries, present_window])
        
        x_img = tf.transpose(x_img, perm=[0,2,1])
        return self.layer(x_img)
    
#         xs = [tf.reshape(x_img[:,i,:], [-1, present_window, 1]) for i in range(nseries)]
#         X = tf.concat(xs, 0)
#         return X
#         ys = []
#         for xval in xs:
#             ys.append(self.layer(xval))
#         return tf.add_n(ys)

In [3]:
def submit(net_model, niter, lr, fname):
    # Start clean
    tf.reset_default_graph()
    sess = tf.Session()

    # Setup placeholders for input and output
    x = tf.placeholder(tf.float32, shape=[None, present_window*nseries], name="x")
    y = tf.placeholder(tf.float32, shape=[None, future_window], name="y")
    
    # Similarly, setup placeholders for dev set
    xdev = tf.placeholder(tf.float32, shape=[None, present_window*nseries], name="xdev")
    ydev = tf.placeholder(tf.float32, shape=[None, future_window], name="ydev")
    
    # Feed forward training and dev examples
#     with tf.variable_scope(net_model, reuse=tf.AUTO_REUSE):
    if net_model is 'CNN':
        nn = CNN([32,64])
    elif net_model is 'FC':
        nn = FC([1024,512])
    elif net_model is 'RNN':
        nn = RNN()

    yhat    = nn(x)
    yhatdev = nn(xdev)

    # Define loss in training and dev set
    with tf.name_scope("loss"):
        train_loss = tf.losses.mean_squared_error(labels=y, predictions=yhat)
        dev_loss = tf.losses.mean_squared_error(labels=ydev, predictions=yhatdev)
        tf.summary.scalar("train_loss", train_loss)
        tf.summary.scalar("dev_loss", dev_loss)

    # Minimize training loss
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(lr).minimize(train_loss)

    # Collect all summaries for TensorBoard
    summ = tf.summary.merge_all()

    # Start of execution
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + fname)
    writer.add_graph(sess.graph)

    for i in range(niter):
        X, Y = make_batch(dftrain, 100)
        Xdev, Ydev = make_batch(dfdev, 100)
        if i % 5 == 0:
            [tloss, dloss, s] = sess.run([train_loss, dev_loss, summ], feed_dict={x: X, y: Y, xdev: Xdev, ydev: Ydev})
            writer.add_summary(s, i)
            writer.flush()
        sess.run(train_step, feed_dict={x: X, y: Y, xdev: Xdev, ydev: Ydev})

In [4]:
net_models = ['CNN','FC']
# net_models = ['RNN']

def main():
    for lr in [1e-2, 1e-3, 1e-4]:
        for net_model in net_models:
            niter = 500
            fname = "{},lr={}".format(net_model, lr)
        
            submit(net_model, niter, lr, fname)
    
main()